In [1]:
import torch
from torch import nn
from torchtext import *
import spacy
import string
import torchtext
import math
import copy
from torch import optim
import torch.nn.functional as F
print(f"Torch version is {torch.__version__} Torchtext version is {torchtext.__version__}")
from pytorch_transformers import OpenAIGPTModel,OpenAIGPTConfig,OpenAIGPTPreTrainedModel
device = torch.device("cuda")
import torch
import json
from tqdm import tqdm


Torch version is 1.2.0 Torchtext version is 0.4.0


In [2]:
# tokenizer_func = lambda s :s.split()
TEXT = data.Field(tokenize ="spacy" ,batch_first=True)
train_data,val_data,test_data = datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_data)
print(f"The vocab size is {TEXT.vocab.stoi.__len__()}")
train_iter = data.BPTTIterator(train_data,batch_size = 64*8 , shuffle=True,bptt_len=50,train=True,device=device)
val_iter = data.BPTTIterator(val_data,batch_size = 64*8 , shuffle=True,bptt_len=50,train=False,device=device)

The vocab size is 33243


In [3]:
config = OpenAIGPTConfig(vocab_size_or_config_json_file=TEXT.vocab.stoi.__len__())

In [4]:
class OpenAIGPTLMHeadModel(OpenAIGPTPreTrainedModel):
    def __init__(self, config):
        super(OpenAIGPTLMHeadModel, self).__init__(config)
        self.transformer = OpenAIGPTModel(config)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.init_weights()
        self.tie_weights()

    def tie_weights(self):
        """ Make sure we are sharing the input and output embeddings.
            Export to TorchScript can't handle parameter sharing so we are cloning them instead.
        """
        self._tie_or_clone_weights(self.lm_head,
                                   self.transformer.tokens_embed)

    def forward(self, input_ids,labels=None):
        transformer_outputs = self.transformer(input_ids)
        hidden_states = transformer_outputs[0]
        lm_logits = self.lm_head(hidden_states)

        outputs = (lm_logits,) + transformer_outputs[1:]
        if labels is not None:
            # Shift so that tokens < n predict n
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)),
                            labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), lm_logits, (all hidden states), (all attentions)
model = OpenAIGPTLMHeadModel(config)
model = torch.nn.DataParallel(model,device_ids=[0,1,2,3,4,5,6,7],output_device=torch.device("cuda:1")).to(device)

In [5]:
import numpy as np

class Stepper():
    "Used to \"step\" from start,end (`vals`) over `n_iter` iterations on a schedule defined by `func`"
    
    def __init__(self, val, n_iter:int, func):
        self.start,self.end = val
        self.n_iter = max(1,n_iter)
        self.func = func
        self.n = 0

    def step(self):
        "Return next value along annealed schedule."
        self.n += 1
        return self.func(self.start, self.end, self.n/self.n_iter)

    @property
    def is_done(self):
        "Return `True` if schedule completed."
        return self.n >= self.n_iter
    
# Annealing functions
def annealing_no(start, end, pct):
    "No annealing, always return `start`."
    return start
  
def annealing_linear(start, end, pct):
    "Linearly anneal from `start` to `end` as pct goes from 0.0 to 1.0."
    return start + pct * (end-start)
  
def annealing_exp(start, end, pct):
    "Exponentially anneal from `start` to `end` as pct goes from 0.0 to 1.0."
    return start * (end/start) ** pct

def annealing_cos(start, end, pct):
    "Cosine anneal from `start` to `end` as pct goes from 0.0 to 1.0."
    cos_out = np.cos(np.pi * pct) + 1
    return end + (start-end)/2 * cos_out


In [6]:
class OneCyclePolicy:
  
    def __init__(self,model , optimizer  ,num_iteration, momentum = (0.95,0.85) , div_factor=25 ,
                 pct_start=0.4,
                 save_model_filename= "hindi_openai_lm_saved_model_weights.pt" ,
                 log_file_name="hindi_openai_lm_train_log.txt",epochs=100,max_lr=0.01):
    
        self.model = model
        self.optimizer = optimizer
        self.criterion=nn.CrossEntropyLoss()
        self.save_model_filename = save_model_filename
        self.log_file_name = log_file_name
        self.num_epochs = epochs

        n = num_iteration * self.num_epochs
        a1 = int(n*pct_start)
        a2 = n-a1
        self.phases = ((a1 , annealing_linear) , (a2 , annealing_cos))
        min_lr = max_lr/div_factor
        self.lr_scheds = self.steps((min_lr,max_lr) , (max_lr,min_lr/1e1))
        self.mom_scheds =self.steps(momentum , momentum[::-1])
        self.idx_s = 0
        self.lr ,self.mom=None,None
        self.update_lr_mom(self.lr_scheds[0].start,self.mom_scheds[0].start)
        self.best_valid_loss = float('inf')

    def logger(self,*args,**kwargs):
        print(*args,**kwargs)
        with open(self.log_file_name,"a") as f:  # appends to file and closes it when finished
            print(file=f,*args,**kwargs)
  
    def steps(self, *steps):
        "Build anneal schedule for all of the parameters."
        return [Stepper(step, n_iter, func=func)for (step,(n_iter,func)) in zip(steps, self.phases)]

    def train_evaluate(self, trainLoader , validLoader ):
        data_loader = {"train":trainLoader , "val":validLoader}
        for epoch in tqdm(range(self.num_epochs),desc="Epochs"):
          
            for phase in ['train', 'val']:

                if phase=="train":     # put the model in training mode
                    self.logger(f"For Epoch [{epoch}|{self.num_epochs}] : ")
                    self.logger(f"\nLearning Rate is {self.lr} momentum is {self.mom}")
                    train_loss = self.train(data_loader[phase])
                    self.logger(f'\nTrain Loss: {train_loss:.3f}')


                else:     # put the model in validation mode
                    valid_loss = evaluate(self.model, data_loader[phase]) 
                    self.logger(f'\nVal. Loss: {valid_loss:.3f}' )

                    if valid_loss < self.best_valid_loss:
                        self.best_valid_loss = valid_loss
                        torch.save(self.model.module.state_dict(), self.save_model_filename)



    def update_lr_mom(self,lr=0.001,mom=0.99):
        for l in self.optimizer.param_groups:
            self.lr , l["lr"]=lr,lr
            if isinstance(self.optimizer , ( torch.optim.Adamax,torch.optim.Adam)):
                  l["betas"] = ( mom, 0.999)
            elif isinstance(self.optimizer, torch.optim.SGD):
                  l["momentum"] =mom
            self.mom = mom


    def train(self, iterator):
        epoch_loss = 0

        self.model.train()

        for batch in iterator:

            self.optimizer.zero_grad()

            text = batch.text
            labels =batch.target
            output  = self.model(text,labels=labels)[0]
            loss = output.mean()
            loss.backward()
            self.optimizer.step()
            self.update_lr_mom(self.lr_scheds[self.idx_s].step() ,self.mom_scheds[self.idx_s].step() )

            if self.lr_scheds[self.idx_s].is_done:
                self.idx_s += 1

            epoch_loss += loss.item()
        return epoch_loss / len(iterator)
                                
def evaluate(model, iterator):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            labels =batch.target
            output  = model(text,labels=labels)[0]
            loss = output.mean()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [8]:
optimizer =  torch.optim.SGD(model.parameters(),lr=1e-3,momentum=True,nesterov=True)
fit_one_cycle = OneCyclePolicy(model ,
                               optimizer ,
                               num_iteration=len(train_iter),epochs=100,pct_start=0.3,max_lr=0.03,div_factor=20)
fit_one_cycle.train_evaluate(train_iter,val_iter)


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

For Epoch [0|100] : 

Learning Rate is 0.0015 momentum is 0.95


/home/ubuntu/instoried/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Train Loss: 8.116

Val. Loss: 6.704


Epochs:   1%|          | 1/100 [03:29<5:45:18, 209.28s/it]

...

Epochs:  98%|█████████▊| 98/100 [5:10:32<06:16, 188.34s/it]


Val. Loss: 4.462
For Epoch [98|100] : 

Learning Rate is 0.00021008366212105692 momentum is 0.949798714699762

Train Loss: 4.299


Epochs:  99%|█████████▉| 99/100 [5:13:39<03:07, 187.95s/it]


Val. Loss: 4.462
For Epoch [99|100] : 

Learning Rate is 0.00016502848187087977 momentum is 0.9499496533270657

Train Loss: 4.299

Val. Loss: 4.461


Epochs: 100%|██████████| 100/100 [5:16:49<00:00, 188.53s/it]


In [ ]:
# model.module.load_state_dict(torch.load("hindi_openai_lm_saved_model_weights.pt"))

In [30]:
import spacy
nlp =spacy.load("en")

In [31]:
def greedy_decode(text,n=20):
    
    word_idx = torch.tensor([TEXT.vocab.stoi[i.text] for i in nlp.tokenizer(text)]).unsqueeze(0).to(device)
    with torch.no_grad():
        for _ in range(n):
            output = model.module(word_idx)[0][:,-1]
            prob = torch.argmax(output,dim=1).unsqueeze(0)
            word_idx = torch.cat([word_idx,prob],dim=1)
    return " ".join([TEXT.vocab.itos[i] for i in word_idx.view(-1)])


In [53]:
greedy_decode("""His greatest ambition was to serve his country """)

'His greatest ambition was to serve his country in the United States . He was a member of the University of London , and was the first of'